In [269]:
import tensorflow as tf
import numpy as np
import math

In [312]:
def distance_matrix(position):
    distance = np.zeros((position.shape[0], position.shape[0]), dtype = np.float32)
    for i in range(position.shape[0]):
        for j in range(position.shape[0]):
            distance[i][j] = (position[i][0] - position[j][0])**2 + (position[i][1] - position[j][1])**2
    return distance



In [441]:
x = tf.placeholder(shape=[None, None], dtype=tf.float32, name="x")
y = tf.placeholder(shape=[None, 1], dtype=tf.float32, name="y")
possition = tf.placeholder(shape=[None,None], dtype=tf.float32, name="possition")


sig1 = tf.Variable(0.1, dtype=tf.float32, name="sig1")
sig2 = tf.Variable(0.1, dtype=tf.float32, name="sig2")
sig3 = tf.Variable(0.1, dtype=tf.float32, name="sig3")
sig4 = tf.Variable(0.1, dtype=tf.float32, name="sig4")


### cov_matrix1 sigma* matrix      xxxxxxxxxxxxxxxxxxxxxx
xxt = tf.matmul(x, tf.transpose(x))
sig1_pow2 = tf.pow(sig1, 2)

cov_matrix1 = sig1_pow2 * xxt


### cov_matrix2 sigma* matrix
def env_pos(x):
    return tf.exp(((-1)*(0.5)*x)/(tf.pow(l,2)))

zzt = tf.map_fn(env_pos, possition)

l = tf.Variable(1, dtype=tf.float32, name="l")
sig2_pow2 = tf.pow(sig2, 2)

cov_matrix2 = sig2_pow2 * zzt 


### cov_matrix3 sigma* matrix
mask = tf.ones_like(xxt) - tf.eye(tf.shape(xxt)[0])
weight = tf.math.multiply(mask, zzt)
zx = tf.matmul(weight, x)
zxxzt = tf.matmul(zx, tf.transpose(zx))
sig3_pow2 = tf.pow(sig3, 2)

cov_matrix3 = sig3_pow2 * zxxzt 


### cov_matrix4 sigma* matrix
noise = tf.eye(tf.shape(xxt)[0])
sig4_pow2 = tf.pow(sig4, 2)

cov_matrix4 = sig4_pow2 * noise


###### main covariance matrix
cov_matrix = cov_matrix1+ cov_matrix2+ cov_matrix3+ cov_matrix4


##### log likelihood term1
cov_det = tf.linalg.det(cov_matrix)
cov_inv = tf.linalg.inv(cov_matrix)
loss_term1 = tf.matmul(tf.matmul(tf.transpose(y), cov_inv), y)

##### log likelihood term2
loss_term2 = tf.log(cov_det)

#### loss function :log likelihood without constant vairable n/2log(pi)
loss = tf.add(loss_term1, loss_term2)

### optimizer
optimizer = tf.train.AdamOptimizer().minimize(loss)

In [435]:


# def env_pos(x):
#     return tf.exp(((-1)*(0.5)*x)/(tf.pow(l,2)))


# possition = tf.Variable(np.asarray([[1, 0], [0, 8]]),dtype=tf.float32)
# sess = tf.Session()
# l = 1.0
# sess.run(possition.initializer)
# zzt = tf.map_fn(env_pos, possition)
# print(sess.run(zzt))


In [436]:
import pandas as pd

##### data

# expressions = pd.read_csv('expressions.txt',sep=" ").loc[0:1]
# expressions = expressions[expressions.columns[0:3]]
# posit = pd.read_csv('positions.txt', sep=",", header=None).loc[0:1]
# print(posit.shape)

expressions = pd.read_csv('expressions.txt',sep=" ")
posit = pd.read_csv('positions.txt', sep=",", header=None)
print(posit.shape)

y1 = expressions['CD20'].values
y1 = y1.reshape(y1.shape[0],1)
# y1.dtype = np.float32
print(y1.shape)
print(y1.dtype)
pos1 = distance_matrix(posit.values)
# pos1.dtype = np.float32
print(pos1.shape)
print(pos1.dtype)
x1 = expressions.drop(columns=['CD20']).values
# x1.dtype = np.float32
print(x1.shape)
print(x1.dtype)


(296, 2)
(296, 1)
float64
(296, 296)
float32
(296, 25)
float64


In [401]:
# posit

In [442]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1):
        a, b, c, d = sess.run([cov_matrix, zzt, cov_det, cov_inv], feed_dict= {x:x1, y:y1, possition: pos1})
        
        
#         b = sess.run(zzt, feed_dict= {x:x1, y:y1, possition: pos1})
#         c = sess.run(cov_matrix3, feed_dict= {x:x1, y:y1, possition: pos1})
#         d = sess.run(cov_matrix4, feed_dict= {x:x1, y:y1, possition: pos1})
#         cv = sess.run(cov_matrix, feed_dict= {x:x1, y:y1, possition: pos1})

#         e = sess.run(x, feed_dict= {x:x1, y:y1, possition: pos1})
#         f = sess.run(loss_term2, feed_dict= {x:x1, y:y1, possition: pos1}) 
#         g = sess.run(cov_det, feed_dict= {x:x1, y:y1, possition: pos1})
#         sess.run(optimizer, feed_dict= {x:x1, y:y1, possition: pos1})
        
#         los = sess.run(loss, feed_dict= {x:x1, y:y1, possition: pos1})
#         sigs = sess.run([sig1, sig2, sig3, sig4], feed_dict={x:x1, y:y1, possition: pos1})
#         print(sigs)
print(d)

[[ 4.6313568e+01 -4.3426723e+00 -5.4564911e-01 ... -7.5140797e-02
   6.3513392e-01  2.0213360e-02]
 [-4.3426466e+00  4.3178734e+01 -5.6341940e-01 ... -7.2242868e-01
   1.1952133e+00 -1.5068620e-01]
 [-5.4566914e-01 -5.6339252e-01  4.7323921e+01 ... -1.1317786e-01
  -5.6536001e-01 -8.2051599e-01]
 ...
 [-7.4284926e-02 -7.2328949e-01 -1.1344336e-01 ...  4.7411011e+01
  -3.4823599e-01  2.3919688e-01]
 [ 6.3547355e-01  1.1951436e+00 -5.6528711e-01 ... -3.4778431e-01
   4.7435535e+01 -7.2218126e-01]
 [ 1.9915814e-02 -1.5057683e-01 -8.2050234e-01 ...  2.3974372e-01
  -7.2289133e-01  3.3488979e+01]]


In [432]:
# print(a.shape)
# print(b.shape)
# print(c.shape)
# print(d.shape)
#print(a[0])
#print(c)
# print(e)
# # print(f.shape)
# print(cv)
# print(g)
# print(cov_inv.shape)
# print(cov_inv)

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
1.0


In [416]:
np.linalg.det(a[0])

inf